# RAG System - Evaluation & Optimization

**Learning Objectives:**
- Measure RAG system quality with metrics
- Evaluate retrieval effectiveness
- Optimize hyperparameters
- Understand trade-offs

**For Interviews:**  
Shows you can:
- Define and measure success metrics
- Systematically improve ML systems
- Make data-driven decisions
- Balance multiple objectives (quality vs cost vs latency)

In [ ]:
import sys
sys.path.append('../src')

import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from rag_pipeline import RAGPipeline

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports complete")

## Part 1: RAG Evaluation Metrics

**Interview Key Point:** You can't improve what you don't measure!

### Key Metrics for RAG Systems:

**Retrieval Metrics:**
- **Precision@K:** Of K retrieved chunks, how many are relevant?
- **Recall@K:** Of all relevant chunks, how many did we retrieve?
- **MRR (Mean Reciprocal Rank):** How high is the first relevant result?

**Generation Metrics:**
- **Accuracy:** Is the answer correct?
- **Groundedness:** Does answer stick to provided context?
- **Relevance:** Does answer address the question?
- **Completeness:** Is all necessary information included?

**System Metrics:**
- **Latency:** Time to generate answer
- **Cost:** LLM API costs
- **Token usage:** Context + generation tokens

## Part 2: Retrieval Evaluation

Let's evaluate how well our retrieval works.

In [ ]:
# Initialize RAG pipeline
print("🚀 Setting up RAG Pipeline...\n")

rag = RAGPipeline(
    collection_name="rag_evaluation",
    chunk_size=400,
    overlap=50
)

# Load knowledge base
rag.add_documents('../data/knowledge_base.json')

print("✓ Pipeline ready!")

In [ ]:
# Load test questions
with open('../data/test_questions.json', 'r') as f:
    test_questions = json.load(f)

print(f"📝 Loaded {len(test_questions)} test questions\n")

# Show example
print("Example test question:")
print(json.dumps(test_questions[0], indent=2))

In [ ]:
# Evaluate retrieval
print("\n📊 Evaluating Retrieval Quality...\n")

metrics = rag.evaluate_retrieval('../data/test_questions.json')

print("\n" + "="*60)
print("RETRIEVAL EVALUATION RESULTS")
print("="*60)
print(f"Precision@3: {metrics['precision@3']:.1%}")
print(f"Recall@3: {metrics['recall@3']:.1%}")
print(f"Questions tested: {metrics['num_questions']}")
print("="*60)

print("\n💡 Interpretation:")
print(f"   • {metrics['precision@3']*100:.0f}% of retrieved chunks were relevant")
print(f"   • We found {metrics['recall@3']*100:.0f}% of all relevant information")

### Understanding Precision vs Recall

**Interview Tip:** Be ready to explain this trade-off!

- **High Precision, Low Recall:** Retrieved chunks are very relevant, but we're missing some good content
  - *Solution:* Increase top_k

- **Low Precision, High Recall:** We're getting all the relevant content, but also irrelevant chunks
  - *Solution:* Decrease top_k, improve chunk quality

- **Both Low:** Retrieval isn't working well
  - *Solutions:* Better embeddings, different chunk size, hybrid search

## Part 3: Hyperparameter Optimization

Let's systematically test different configurations.

In [ ]:
# Test different chunk sizes
print("🔬 Testing Different Chunk Sizes...\n")

chunk_sizes = [200, 300, 400, 600]
results = []

for chunk_size in chunk_sizes:
    print(f"Testing chunk_size={chunk_size}...")
    
    # Create pipeline with this chunk size
    test_rag = RAGPipeline(
        collection_name=f"test_chunk_{chunk_size}",
        chunk_size=chunk_size,
        overlap=50
    )
    
    # Add documents
    test_rag.add_documents('../data/knowledge_base.json')
    
    # Evaluate
    metrics = test_rag.evaluate_retrieval('../data/test_questions.json')
    
    results.append({
        'chunk_size': chunk_size,
        'precision': metrics['precision@3'],
        'recall': metrics['recall@3'],
        'f1_score': 2 * (metrics['precision@3'] * metrics['recall@3']) / 
                   (metrics['precision@3'] + metrics['recall@3'] + 1e-10),
        'total_chunks': test_rag.vector_store.count()
    })
    
    # Cleanup
    test_rag.vector_store.delete_collection()

results_df = pd.DataFrame(results)
print("\n" + results_df.to_string(index=False))

In [ ]:
# Visualize chunk size impact
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Precision & Recall
axes[0].plot(results_df['chunk_size'], results_df['precision'], 
             marker='o', label='Precision@3', linewidth=2)
axes[0].plot(results_df['chunk_size'], results_df['recall'], 
             marker='s', label='Recall@3', linewidth=2)
axes[0].plot(results_df['chunk_size'], results_df['f1_score'], 
             marker='^', label='F1 Score', linewidth=2, linestyle='--')
axes[0].set_xlabel('Chunk Size (words)', fontweight='bold')
axes[0].set_ylabel('Score', fontweight='bold')
axes[0].set_title('Retrieval Quality vs Chunk Size', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Total chunks (cost proxy)
axes[1].bar(results_df['chunk_size'].astype(str), results_df['total_chunks'], 
            edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Chunk Size (words)', fontweight='bold')
axes[1].set_ylabel('Total Chunks', fontweight='bold')
axes[1].set_title('Storage/Embedding Cost vs Chunk Size', fontweight='bold')

plt.tight_layout()
plt.show()

# Find best chunk size
best_idx = results_df['f1_score'].idxmax()
best_chunk_size = results_df.loc[best_idx, 'chunk_size']
print(f"\n⭐ Best chunk size: {best_chunk_size} words (F1={results_df.loc[best_idx, 'f1_score']:.3f})")

## Part 4: top_k Optimization

**Interview Key Point:** top_k affects quality AND cost!

In [ ]:
# Test different top_k values
print("🔬 Testing Different top_k Values...\n")

# Reinitialize with best chunk size
eval_rag = RAGPipeline(
    collection_name="eval_topk",
    chunk_size=400,  # Using default
    overlap=50
)
eval_rag.add_documents('../data/knowledge_base.json')

topk_results = []

for k in [1, 2, 3, 4, 5]:
    print(f"Testing top_k={k}...")
    
    # Manually evaluate with this top_k
    precision_scores = []
    recall_scores = []
    
    for item in test_questions:
        question = item['question']
        expected_docs = set(item['expected_doc_ids'])
        
        # Retrieve with this top_k
        results = eval_rag.retrieve(question, top_k=k)
        
        # Get retrieved doc IDs
        retrieved_docs = set()
        for meta in results['metadatas']:
            retrieved_docs.add(meta['source_doc_id'])
        
        # Calculate metrics
        relevant = retrieved_docs & expected_docs
        precision = len(relevant) / len(results['ids']) if results['ids'] else 0
        recall = len(relevant) / len(expected_docs) if expected_docs else 0
        
        precision_scores.append(precision)
        recall_scores.append(recall)
    
    avg_precision = sum(precision_scores) / len(precision_scores)
    avg_recall = sum(recall_scores) / len(recall_scores)
    
    # Estimate cost (tokens sent to LLM)
    # Assuming ~400 words per chunk, ~1.3 tokens per word
    est_tokens = k * 400 * 1.3
    
    topk_results.append({
        'top_k': k,
        'precision': avg_precision,
        'recall': avg_recall,
        'f1_score': 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall + 1e-10),
        'est_tokens': int(est_tokens)
    })

topk_df = pd.DataFrame(topk_results)
print("\n" + topk_df.to_string(index=False))

# Cleanup
eval_rag.vector_store.delete_collection()

In [ ]:
# Visualize top_k trade-offs
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Quality metrics
axes[0].plot(topk_df['top_k'], topk_df['precision'], marker='o', label='Precision', linewidth=2)
axes[0].plot(topk_df['top_k'], topk_df['recall'], marker='s', label='Recall', linewidth=2)
axes[0].plot(topk_df['top_k'], topk_df['f1_score'], marker='^', label='F1', linewidth=2, linestyle='--')
axes[0].set_xlabel('top_k', fontweight='bold')
axes[0].set_ylabel('Score', fontweight='bold')
axes[0].set_title('Quality vs top_k', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Cost (token usage)
axes[1].bar(topk_df['top_k'].astype(str), topk_df['est_tokens'], edgecolor='black', alpha=0.7, color='coral')
axes[1].set_xlabel('top_k', fontweight='bold')
axes[1].set_ylabel('Context Tokens', fontweight='bold')
axes[1].set_title('LLM Cost vs top_k', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Insight: Higher top_k improves recall but increases cost linearly!")

## Part 5: Quality-Cost Trade-off Analysis

In [ ]:
# Calculate cost-efficiency score
# Higher F1 with fewer tokens = better
topk_df['efficiency'] = topk_df['f1_score'] / (topk_df['est_tokens'] / 1000)

print("\n📊 COST-EFFICIENCY ANALYSIS\n")
print("="*60)
print(topk_df[['top_k', 'f1_score', 'est_tokens', 'efficiency']].to_string(index=False))
print("="*60)

# Find sweet spot
best_efficiency_idx = topk_df['efficiency'].idxmax()
print(f"\n⭐ Most cost-efficient: top_k={topk_df.loc[best_efficiency_idx, 'top_k']}")
print(f"   F1 Score: {topk_df.loc[best_efficiency_idx, 'f1_score']:.3f}")
print(f"   Est. Tokens: {topk_df.loc[best_efficiency_idx, 'est_tokens']}")

## Part 6: Recommended Configurations

Based on our evaluation, here are recommended configs for different use cases:

In [ ]:
print("\n⚙️ RECOMMENDED CONFIGURATIONS\n")
print("="*60)

configs = [
    {
        'use_case': 'Cost-Optimized',
        'chunk_size': 300,
        'overlap': 30,
        'top_k': 2,
        'when': 'High query volume, tight budget',
        'tradeoff': 'Lower quality, minimal cost'
    },
    {
        'use_case': 'Balanced (Recommended)',
        'chunk_size': 400,
        'overlap': 50,
        'top_k': 3,
        'when': 'Most production use cases',
        'tradeoff': 'Good quality, moderate cost'
    },
    {
        'use_case': 'Quality-Optimized',
        'chunk_size': 600,
        'overlap': 75,
        'top_k': 5,
        'when': 'Critical applications, quality > cost',
        'tradeoff': 'Best quality, highest cost'
    },
    {
        'use_case': 'Low-Latency',
        'chunk_size': 200,
        'overlap': 20,
        'top_k': 2,
        'when': 'Real-time chat, fast response needed',
        'tradeoff': 'Faster, but less context'
    }
]

for config in configs:
    print(f"\n{config['use_case']}:")
    print(f"  chunk_size: {config['chunk_size']}, overlap: {config['overlap']}, top_k: {config['top_k']}")
    print(f"  When: {config['when']}")
    print(f"  Trade-off: {config['tradeoff']}")

print("\n" + "="*60)

## Part 7: Improvement Strategies

**Interview talking points for improving RAG systems:**

In [ ]:
print("\n🚀 RAG OPTIMIZATION STRATEGIES\n")
print("="*60)

strategies = [
    {
        'category': 'Better Retrieval',
        'techniques': [
            '• Hybrid search (combine keyword BM25 + semantic)',
            '• Reranking: Retrieve top 10, rerank to top 3',
            '• Query expansion (paraphrase user query)',
            '• Domain-specific embedding models',
            '• Metadata filtering (filter by date, category, etc.)'
        ]
    },
    {
        'category': 'Better Chunking',
        'techniques': [
            '• Semantic chunking (split at topic boundaries)',
            '• Sentence-aware chunking (don\'t break mid-sentence)',
            '• Parent-child chunking (small chunks + larger context)',
            '• Include chunk context (add doc title to each chunk)'
        ]
    },
    {
        'category': 'Better Generation',
        'techniques': [
            '• Few-shot examples in prompt',
            '• Chain-of-thought reasoning',
            '• Ask LLM to cite sources',
            '• Self-consistency (generate multiple, pick best)',
            '• Fine-tune LLM on domain data'
        ]
    },
    {
        'category': 'System Improvements',
        'techniques': [
            '• Cache frequent queries',
            '• Async/streaming responses',
            '• Monitor and log all queries',
            '• A/B test different configurations',
            '• Collect user feedback for improvement'
        ]
    }
]

for strategy in strategies:
    print(f"\n{strategy['category']}:")
    for technique in strategy['techniques']:
        print(f"  {technique}")

print("\n" + "="*60)

## Summary: Evaluation Best Practices

### What You've Learned:

✅ **Metrics:**
- Precision, Recall, F1 for retrieval quality
- Cost metrics (tokens, latency)
- How to measure quality vs cost trade-offs

✅ **Optimization:**
- Systematic hyperparameter tuning
- Finding the right chunk size
- Balancing top_k for quality and cost

✅ **Interview Ready:**
- Can explain evaluation methodology
- Understand precision/recall trade-offs
- Know multiple improvement strategies
- Can discuss production considerations

### Key Interview Talking Points:

1. **"How do you evaluate a RAG system?"**
   - Retrieval metrics (precision/recall)
   - Generation metrics (accuracy/groundedness)
   - System metrics (latency/cost)
   - Use test sets with known relevant documents

2. **"How do you optimize chunk size?"**
   - Evaluate on test set with different sizes
   - Consider document structure and domain
   - Balance precision (small chunks) vs context (large chunks)
   - Typical range: 200-600 words

3. **"How do you handle quality vs cost trade-offs?"**
   - Define acceptable quality threshold
   - Minimize cost while meeting threshold
   - Use smaller top_k for common queries
   - Cache results for repeated questions
   - Consider streaming for better UX

4. **"How would you improve a poorly performing RAG system?"**
   - First, measure current performance
   - Identify bottleneck (retrieval or generation?)
   - Try hybrid search, reranking, better embeddings
   - Improve prompts and chunking strategy
   - A/B test changes with real users

In [ ]:
# Cleanup
print("\n🧹 Cleaning up...")
rag.vector_store.delete_collection()
print("✓ Done!")

print("\n" + "="*60)
print("RAG EVALUATION & OPTIMIZATION COMPLETE! 🎉")
print("="*60)
print("\nYou now have a production-ready RAG system!")
print("\nNext steps:")
print("1. Review evaluation.md for detailed metrics")
print("2. Read interview_questions.md to prepare for interviews")
print("3. Practice explaining this system to others")
print("4. Try building RAG for your own use case!")